In [ ]:
# Install missing Python packages in Google Colab
!pip install ta
!pip install prophet
!pip install pykalman
!pip install PyWavelets

# Gather data

In [ ]:
from data.data_collection import gather_data

startDateStr = '2008-10-01'
endDateStr = '2018-10-02' # documentation said that endDateStr is exclusive for both yahoofinance and the original code, but actually printing the shapes showed otherwise..
instrumentIds = list(set(['ITOT', 'ACWI', 'IWV', 'VT', 'VTI',
                'DIA', 'RSP', 'IOO', 'IVV', 'SPY',
                'SHE', 'IWM', 'OEF', 'QQQ',
                'CVY', 'RPG', 'RPV', 'IWB', 'IWF',
                'IWD', 'IVW', 'IVE', 'PKW',
                'PRF', 'SDY', 'VV', 'VUG',
                'VTV', 'MGC', 'MGK', 'MGV', 'VIG',
                'VYM', 'DTN', 'DLN', 'MDY', 'DVY',
                'IWR', 'IWP', 'IWS', 'IJH', 'IJK',
                'IJJ', 'PDP', 'DON', 'IWC', 'IWM',
                'IWO', 'IWN', 'IJR', 'IJT', 'IJS',
                'EEB', 'IDV', 'ACWX', 'BKF', 'EFA',
                'EFG', 'EFV', 'SCZ', 'EEM', 'PID',
                'DWX', 'DEM', 'DGS', 'AAXJ', 'EZU',
                'EPP', 'IEV', 'ILF', 'FEZ', 'VGK',
                'VPL', 'DFE', 'EWA', 'EWC', 'EWG',
                'EWI', 'EWJ', 'EWD', 'EWL', 'EWP',
                'EWU', 'DXJ', 'EWZ', 'FXI', 'EWH',
                'EWW', 'RSX', 'EWS', 'EWM','EWY',
                'EWT', 'EPI', 'XLY', 'IYC', 'ITB',
                'XHB', 'VCR','XLP', 'IYK', 'VDC',
                'XLE', 'IYE', 'IGE',
                'VDE', 'QCLN', 'XLF','IYF', 'KBE',
                'KRE', 'VFH']))


data = gather_data(startDateStr, endDateStr, instrumentIds)
data_close = data['close']
data_open = data['open']
data_high = data['high']
data_low = data['low']
data_vol = data['vol']

[*********************100%***********************]  111 of 111 completed


# Prepocessing
### Filtering ETFs for liquidity

In [ ]:
data['yfinance_formatted'][['QQQ', 'IJJ']].xs('Volume', level='Price', axis=1)


# Find Co-Integrated Pairs

First, let's try to estimate the time we need.

In [47]:
from enum import unique


n_cols = len(data['yfinance_formatted'].columns.get_level_values('Ticker').unique())
max_cols = 50
if n_cols > max_cols and max_cols > 0:
    cols = data['yfinance_formatted'].columns.get_level_values('Ticker').unique()[:max_cols]
    n_cols = max_cols
else:
    cols = data['yfinance_formatted'].columns.get_level_values('Ticker').unique()
first_n_cols = data['yfinance_formatted'].columns.get_level_values('Ticker').unique()[:n_cols]
# on average, second per pair is 
seconds_per_pair = 63.06 / 190 # found empirically, can change
# combinatorially find number of unique pairs of columns
unique_pairs = n_cols * (n_cols - 1) / 2 
expected_time = unique_pairs * seconds_per_pair
print(f"Expected time to run all pairs: {expected_time / 60:.2f} minutes")

Expected time to run all pairs: 6.78 minutes


In [ ]:
%load_ext autoreload
%autoreload 2

from preprocessing.cointegration import find_cointegrated_pairs
from utils.visualization import plot_heatmap

# First, we'll try it for a smaller (random) subset of the data
import time
start_time = time.time()
scores, pvalues, pairs = find_cointegrated_pairs(data['yfinance_formatted'][first_n_cols])
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")
pairs_data = {key:value[1]  for (key, value) in pairs.items()}
pairs_data = sorted(pairs_data.items(), key=lambda x: x[1])
pairs_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Completed 190 pairs
Execution time: 63.06 seconds


[(('SCZ', 'KBE'), 0.0006297952518477276),
 (('KBE', 'IWM'), 0.0027222425205743446),
 (('VPL', 'KBE'), 0.0028637215258346233),
 (('RSP', 'IYF'), 0.0029475246628193306),
 (('RSP', 'MGV'), 0.0044175344736164385),
 (('RPG', 'IYF'), 0.004688453320363014),
 (('KBE', 'RPG'), 0.005025586550337979),
 (('KBE', 'RSP'), 0.005931532077904926),
 (('IWB', 'KBE'), 0.007738880430232023),
 (('MGC', 'KBE'), 0.01021720456129566),
 (('KBE', 'IYF'), 0.01278227219644674),
 (('MGV', 'IWM'), 0.012794252697356893),
 (('QQQ', 'KBE'), 0.012954083682456628),
 (('ACWI', 'IWM'), 0.01565197884049768),
 (('IWB', 'RPG'), 0.01622617221861165),
 (('ACWI', 'IYF'), 0.01734074519818347),
 (('RPG', 'MGV'), 0.017467749200500187),
 (('ACWI', 'KBE'), 0.019711361216843235),
 (('MGV', 'IYF'), 0.02552766745186497),
 (('ITOT', 'KBE'), 0.029399392061927522),
 (('IWB', 'MGV'), 0.03115675941506319),
 (('ITOT', 'MGV'), 0.032253708453548),
 (('IWM', 'IYF'), 0.03313020169751914),
 (('IWB', 'IYF'), 0.03382703480498584),
 (('KBE', 'MGV'), 